In [2]:
import sys
import os

from shapely.geometry import LineString, MultiLineString
from shapely.geometry import LinearRing, Point
from shapely import GeometryCollection
import matplotlib.pyplot as plt
from pyproj import Transformer
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import pickle
%matplotlib inline

In [3]:
with open("lat_lon_dict.pkl", 'rb') as file:
    lat_lon_dict = pickle.load(file)
    

ac_data = []
with open("results/test2/train_aircraft_data.pkl", 'rb') as file:
    
    while True:
        try:
            ac_data.append(pickle.load(file))
        
        except:
            break
            
            
old_data = []
with open("results/test/train_aircraft_data.pkl", 'rb') as file:
    
    while True:
        try:
            old_data.append(pickle.load(file))
        
        except:
            break
            
            

In [4]:
def process_aircraft_data(idx, ac_data):

    idx = 0

    aircraft_data = ac_data[idx]

    nmac_lat, nmac_lon, nmac_count = [], [], []

    for worker_data in aircraft_data:
        for aircraft_id in worker_data:

            aircraft_data = worker_data[aircraft_id]

            NMAC = np.array(aircraft_data["NMAC"]) == 1

            lat_nmac = np.array(aircraft_data["Lat"])
            lon_nmac = np.array(aircraft_data["Lon"])

            lat_nmac = lat_nmac[NMAC]
            lon_nmac = lon_nmac[NMAC]

            nmac_lat += list(lat_nmac)
            nmac_lon += list(lon_nmac)
            nmac_count += [1]*len(lat_nmac)
            
    return nmac_lat, nmac_lon, nmac_count

# Plot Linestrings on Map

In [5]:
lats = []
lons = []
names = []

for route in lat_lon_dict:
    n_coords = len(lat_lon_dict[route])
    
    names = names + [route]*n_coords + [None]
    
    coords = list(zip(*lat_lon_dict[route]))
    
    lats = lats + list(coords[0]) + [None]
    lons = lons + list(coords[1]) + [None]

In [6]:
lat_lon_dict['I30L']

[(32.9040296356683, -96.4574314470633),
 (32.8992256589177, -96.4824768014311),
 (32.8710261629565, -96.5426339836454),
 (32.864313784922, -96.5561280426337),
 (32.8602084976526, -96.5645875628405),
 (32.8383069908968, -96.6131511092119),
 (32.8339514747531, -96.6199578445428),
 (32.8306719494872, -96.6255920569911),
 (32.8228177541351, -96.6395186534877),
 (32.8072392427546, -96.6712581926414),
 (32.8048438200859, -96.6761922416459),
 (32.8020586161443, -96.6839982990674),
 (32.7998779625346, -96.6921298467542),
 (32.7991940306619, -96.7149628392928),
 (32.7988715849554, -96.7243212694816),
 (32.7940711055734, -96.7683439899531),
 (32.7868825919309, -96.782888275998),
 (32.7797732451483, -96.786856327031),
 (32.7745047657453, -96.8196434221632),
 (32.7729170806359, -96.8419899998559),
 (32.7763467978987, -96.873164784),
 (32.768452662434, -96.9097236924722)]

In [6]:
# p = Point([-96.7,32.90]).buffer(0.01)
# x, y = p.boundary.xy
# x = np.array(x)
# y = np.array(y)

In [7]:
fig = px.line_mapbox(lat=lats, lon=lons, hover_name=names,mapbox_style="stamen-terrain")

# data2 = go.Scattermapbox(lon=x, lat=y,mode='lines',marker=go.scattermapbox.Marker(
#          size=14, color="BlueViolet"
#      ))
#fig.add_trace(data2)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': '<b>%{hovertext}</b><br><br>lat=%{lat}<br>lon=%{lon}<extra></extra>',
              'hovertext': array(['I30L', 'I30L', 'I30L', ..., 'I35L', 'I35L', None], dtype=object),
              'lat': array([32.90402964, 32.89922566, 32.87102616, ..., 32.79050425, 32.78373905,
                                    nan]),
              'legendgroup': '',
              'line': {'color': '#636efa'},
              'lon': array([-96.45743145, -96.4824768 , -96.54263398, ..., -96.84095204,
                            -96.82288361,          nan]),
              'mode': 'lines',
              'name': '',
              'showlegend': False,
              'subplot': 'mapbox',
              'type': 'scattermapbox'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'mapbox': {'center': {'lat': 32.9264897995645, 'lon': -96.76179285411528},
                          'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
                          'style': 'stamen-terrain',
                          'zoom': 8},
               'margin': {'t': 60},
               'template': '...'}
})

In [10]:
nmac_lat_old, nmac_lon_old, nmac_count_old = process_aircraft_data(0,old_data)


df1 = pd.DataFrame(data=[nmac_lat_old,nmac_lon_old,nmac_count_old]).T
df1.rename(columns={0:'lat', 1:'lon', 2:'NMAC'},inplace=True)

df1 = df1.groupby([df1["lat"].round(3), df1["lon"].round(3)]).agg(
                {"NMAC": "sum"}).reset_index().sort_values("NMAC", ascending=False)
        
    
fig = px.density_mapbox(df1, lat='lat', lon='lon', z='NMAC',
                        mapbox_style="stamen-terrain", range_color=[0,df1.NMAC.max()],width=800, height=600 )

data2 = px.line_mapbox(lat=lats, lon=lons, hover_name=names,mapbox_style="stamen-terrain")

fig.add_trace(data2.data[0])


In [11]:
fig.write_html("early_training.html")

In [12]:
nmac_lat, nmac_lon, nmac_count = process_aircraft_data(0,ac_data)


df = pd.DataFrame(data=[nmac_lat,nmac_lon,nmac_count]).T
df.rename(columns={0:'lat', 1:'lon', 2:'NMAC'},inplace=True)

df = df1.groupby([df["lat"].round(4), df["lon"].round(4)]).agg(
                {"NMAC": "sum"}).reset_index().sort_values("NMAC", ascending=False)
        
    
fig = px.density_mapbox(df, lat='lat', lon='lon', z='NMAC',
                        mapbox_style="stamen-terrain", range_color=[0,df1.NMAC.max()],width=800, height=600 )

data2 = px.line_mapbox(lat=lats, lon=lons, hover_name=names,mapbox_style="stamen-terrain")

fig.add_trace(data2.data[0])


In [13]:
fig.write_html("converged.html")

In [10]:
idx = -1

aircraft_data = ac_data[idx]

lat,lon, nmac, simt = [], [], [], []

for worker_data in aircraft_data:
    for aircraft_id in worker_data:

        aircraft_data = worker_data[aircraft_id]

        lat += aircraft_data["Lat"] + [None]
        lon += aircraft_data["Lon"] + [None]
        nmac += aircraft_data["NMAC"] + [None]
        simt += aircraft_data["time"] + [None]


In [49]:


df = pd.DataFrame(data=[lat,lon,nmac,simt]).T
df.rename(columns={0:'lat', 1:'lon', 2:'nmac',3:'simt'},inplace=True)

fig = px.scatter_mapbox(df, lat="lat", lon="lon", animation_frame="simt",mapbox_style="stamen-terrain")

data2 = px.line_mapbox(lat=lats, lon=lons, hover_name=names,mapbox_style="stamen-terrain")

fig.add_trace(data2.data[0])
# fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 30
# fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500

#fig.update_geos(projection_type="equirectangular", visible=True, resolution=110)


ValueError: Value of 'color' is not the name of a column in 'data_frame'. Expected one of ['lat', 'lon', 'nmac', 'simt'] but received: red